# 模型推理 - 使用手动制作训练集进行 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/home/wangfuxin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True, # 使用4位量化
                              bnb_4bit_quant_type='nf4', # 量化类型
                              bnb_4bit_use_double_quant=True, # 使用双量化
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16']) # 计算数据类型

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto', # 自动分配设备
                                      trust_remote_code=True, # 信任远程代码
                                      revision='b098244') 

/home/wangfuxin/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wangfuxin/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]
/home/wangfuxin/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)
print("原始模型回答：", response)

原始模型回答： 乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、刚强、积极向上等含义。在八宫图（后天八卦）中，乾卦位于北方，与事业、领导、刚强、坚定等有关。

乾卦的含义可以从以下几个方面来解释：

1. 天：乾卦象征着天，代表着宇宙、大自然、力量和刚强。它反映了天地间规律的运行，以及事物发展变化的规律。

2. 干燥：乾卦所代表的干燥，并不是指干燥无水，而是指万物生长过程中所需的水分已经足够，不再需要补充。乾卦象征着事物的成熟和固定。

3. 刚强：乾卦代表着刚强，象征着事物发展过程中所表现出的坚定、积极向上的精神。它告诉我们在面对困难时，要有坚定的信念，勇往直前。

4. 积极向上：乾卦象征着积极向上，代表着事物发展过程中所展现出的活力和创造力。它鼓励我们在追求目标时，要保持热情，勇往直前，不断努力。

总之，乾卦是《易经》中的第一卦，它象征着天、干燥、刚强、积极向上等含义。在实际生活中，乾卦告诉我们面对困难时要坚定信念，勇往直前，保持积极向上的心态。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [7]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

周易中的讼卦（又称法卦）是《易经》中的第二卦，位于后天八卦的西方。讼卦由两个阳爻夹一个阴爻构成，象征着诉讼、争端、矛盾、法则等含义。在八宫图（后天八卦）中，讼卦位于西方，与法律、争端、制度、法则等有关。

讼卦的含义可以从以下几个方面来解释：

1. 诉讼：讼卦象征着诉讼，代表着人们在解决争端、纠纷时的法律诉讼。它反映了人们在面对矛盾和冲突时，所采取的诉讼和争斗方式。

2. 争端：讼卦所代表的争端，是指人们在争夺利益、地位、权力等过程中所引发的各种矛盾和冲突。它告诉我们在处理争端时要理性、公平，遵循法律法规。

3. 矛盾：讼卦反映了人们在现实生活中所遇到的矛盾和问题。它告诉我们面对矛盾时要善于调节，通过沟通和协商解决问题，避免矛盾升级。

4. 法则：讼卦象征着法则和制度，代表着人们在处理争端和矛盾时要遵循的法律法规、道德准则等。它要求我们在解决问题时，要合法合规，遵循社会秩序。

总之，讼卦是《易经》中的第二卦，它象征着诉讼、争端、矛盾、法则等含义。在实际生活中，讼卦告诉我们面对争端时要理性、公平，善于沟通协商，遵循法律法规和道德准则。


## 使用微调后的 ChatGLM3-6B

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [8]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [9]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query) # 原始模型回答

    inputs = tokenizer(query, return_tensors="pt").to(0) # 将输入移动到GPU 0
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512) # 微调后模型生成
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True) # 解码生成的输出
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response # 返回原始和微调后的回答

### 微调前后效果对比

In [10]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，卦象为天。乾卦象征着刚强，强盛，高，万物的始， Iconia。乾卦由两个阴爻夹一个阳爻构成，表现了天的生活，即刚强的生活。

乾卦的卦辞：

乾，元，亨，利，贞。

乾卦的卦辞说明：

乾，象征刚强，强盛，高，万物的始。

亨，通，顺利。

利，有利，吉。

贞，正，吉。

乾卦的卦辞表明了刚强、强盛、高、为始万物的情况，是一种通、吉、正的情况。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是乾元的卦，乾元是指天，它反映了天的基本性质和功能。乾卦是由两个阴爻夹一个阳爻构成，象征天，也象征君子，它反映了君子应该具有的品德和行为。阳爻表示阳，阴爻表示阴，阴爻在阳爻夹持下，阳爻不能发挥功能，所以乾卦象征天，也象征君子。


In [11]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [12]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（讼卦：象征诉讼）为乾卦下，坤卦上。在坤卦之上加入了乾卦，象征刚毅的君子在柔顺的环境中表现出毅力和坚持，但过于刚硬可能会引起争端。讼卦警告君子要柔软谦逊，以和为贵。

卦象：上坤下乾

卦义：诉讼、争端、矛盾

指数：卦数 Counter 卦

断卦：

诉诸 court 是在柔顺中坚持刚毅，但过于刚硬会引争端。君子应知错就改，柔软谦逊，以和为贵。

extend-image：

下坤上乾 

坎上坤下乾 

下坤上乾 

坤上乾下 

下坤上乾


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
